In [1]:
import torch
from tqdm import tqdm
from torch.nn.functional import normalize
import numpy as np
import clip
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import cv2
from base64 import b64encode, b64decode
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

feature_dir = './data-engine/precomputed_features'
clip_feature = np.load(f'{feature_dir}/existing-data-clip-feature.npy')
color_feature = np.load(f'{feature_dir}/existing-data-clip-feature.npy')

all_features = clip_feature
all_features = np.concatenate([clip_feature, color_feature], axis=1)

In [2]:
class ChartDataset(Dataset):
    def __init__(self, img_dir, img_paths, all_features):
        self.img_dir = img_dir
        self.img_paths = img_paths
        self.features = all_features
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = f"{self.img_dir}/{self.img_paths[idx]}"
        features = self.features[idx]
        return features, img_path, idx  # Return the filename as well

img_dir = '~/LLaVA/playground/data/unichart-pretrain-data/content/images'
data = pd.read_csv('~/LLaVA/data-engine/precomputed_features/existing-data-imgnames.csv')
img_paths = data['imgname']
dataset = ChartDataset(img_dir, img_paths, all_features)
dataset_size, emb_size = all_features.shape
print(dataset_size, emb_size)

611934 512


In [3]:
from torch.nn.functional import normalize

path_str_type = 'U200'  # 假设路径字符串最长为200个字符

save_dir = r'./sample_files'

# Setting up memory-mapped files
emb_memory_loc = f'{save_dir}/embeddings_memmap.dat'
paths_memory_loc = f'{save_dir}/paths_memmap.dat'

emd_memmap = np.memmap(emb_memory_loc, dtype='float32', mode='w+', shape=(dataset_size, emb_size))
paths_memmap = np.memmap(paths_memory_loc, dtype=path_str_type, mode='w+', shape=(dataset_size,))


In [4]:
dataloader = DataLoader(dataset, batch_size=10240, shuffle=False, num_workers=8)
for features, path, idx in tqdm(dataloader):
        if any(i >= dataset_size for i in idx):
            raise IndexError(f"Index {idx} out of range for dataset size {dataset_size}")
        
        features = torch.tensor(features, dtype=torch.float32)
        emd_memmap[idx] = normalize(features.unsqueeze(0), dim=1).cpu()
        paths_memmap[idx] = path


  0%|          | 0/60 [00:00<?, ?it/s]/tmp/ipykernel_2217/2031757989.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features, dtype=torch.float32)
100%|██████████| 60/60 [01:19<00:00,  1.33s/it]


In [5]:
import yaml
import random
import numpy as np
import logging
from clustering import compute_centroids

logger = logging.getLogger(__name__) 
logger.addHandler(logging.StreamHandler())

# confg_file = "configs/openclip/clustering_configs.yaml"
# ## -- Load kmeans clustering parameters from configs file
# with open(confg_file, 'r') as y_file:
#     params = yaml.load(y_file, Loader=yaml.FullLoader)

## -- Fix the seed

SEED = 1234
random.seed(SEED)
emb_memory_loc = f'{save_dir}/embeddings_memmap.dat'
paths_memory_loc = f'{save_dir}/paths_memmap.dat'
dataset_size, emb_size = all_features.shape
path_str_type = "U200"

emb_memory = np.memmap(emb_memory_loc, dtype='float32', mode='r', shape=(dataset_size, emb_size))
paths_memory = np.memmap(paths_memory_loc, dtype=path_str_type, mode='r', shape=(dataset_size,))

save_folder = f'{save_dir}/kmeans'
compute_centroids(
    data=emb_memory,
    ncentroids=100,
    niter=100,
    seed=SEED,
    Kmeans_with_cos_dist=True,
    save_folder=save_folder,
    logger=logger,
    verbose=True,
)

Running Kmeans clustering using faiss on dataset of shape (611934, 512) ....
Kmeans parameters: {'data': memmap([[-0.01660688, -0.01851731,  0.01089192, ...,  0.00799155,
          0.0202777 , -0.00223273],
        [ 0.00667938, -0.01185627,  0.00756514, ...,  0.00679741,
          0.0085193 ,  0.03099752],
        [-0.02928183, -0.00341169, -0.00028369, ...,  0.0108927 ,
          0.00254303,  0.01811059],
        ...,
        [-0.01029516,  0.00058201, -0.0117059 , ...,  0.00969967,
          0.01243755, -0.00898463],
        [-0.02057116, -0.00605767, -0.01089924, ...,  0.00559856,
          0.01510197, -0.01349618],
        [-0.01501139,  0.00198137, -0.00944601, ...,  0.00906353,
          0.02739928, -0.01194653]], dtype=float32), 'ncentroids': 100, 'niter': 100, 'seed': 1234, 'Kmeans_with_cos_dist': True, 'save_folder': './630/kmeans', 'logger': <Logger __main__ (WARNING)>, 'verbose': True} ....
Clustering on cpu ....
Sampling a subset of 25600 / 611934 for training
Clustering 2

In [ ]:
from sort_clusters import assign_and_sort_clusters

    
assign_and_sort_clusters(
    data=emb_memory,
    paths_list=paths_memory,
    sim_metric='cosine',
    keep_hard=True,
    kmeans_with_cos_dist=True,
    save_folder= f'{save_dir}/kmeans',
    sorted_clusters_file_loc=f'{save_dir}/sorted_clusters',
    cluster_ids=range(0, 100),
    logger=logger,
) 

In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import math
import os
import numpy as np
import pandas as pd
import submitit
import torch
from tqdm import tqdm
import pickle
import random
import math
import time
import pprint

# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

IMAGE_NAME_INDEX = 0
IMAGE_ID_IN_CLUSTER_INDEX = 1
DIST_METRIC_INDEX = 2
CLUSTER_NUM_INDEX = 3


FINAL_EPS = {
    0.059: 0.5,
    0.060: 0.5,
    0.061: 0.5,
    0.062: 0.5,
    0.063: 0.5,
    0.065: 0.5,
    0.07: 0.5,
}

# FINAL_EPS = {
#     0.00095: 0.796314,
#     0.0395: 0.701809,
#     0.03: 0.720969,
#     0.07: 0.633638,
#     0.083: 0.602154,
#     0.124: 0.499306,
#     0.165: 0.397715,
#     0.205: 0.305327,
#     0.26: 0.197576,
# }



def init_memmap_embs(
    embs_memory_loc: str, dataset_size: int, emd_size: int = 512, dtype: str = "float32"
) -> np.memmap:
    """
    Initializes a memory-mapped NumPy array to read embeddings of examples.

    Args:
        embs_memory_loc (str): Path to the memory-mapped file.
        dataset_size (int): Size of the dataset.

        emd_size (int): Dimensionality of the embeddings.
        dtype (str): Data type of the embeddings.

    Returns:
        np.memmap: A memory-mapped NumPy array.
    """
    embs = np.memmap(
        embs_memory_loc, dtype=dtype, mode="r", shape=(dataset_size, emd_size)
    )
    return embs


class SemDeDupJob(submitit.helpers.Checkpointable):
    """
    - Each SLURMJob will run SemDeDup on number of clusters and save dataframe with which examples to keep from each cluster.
    - Parallelize job_start_cluster across jobs so that preemption in the middle of an epoch isn't a problem and because we want to
    keep the shard structure anyway.
    - Process more than one cluster per job=> run multiple taks inside each jobs.
    - Preempted jobs get resubmitted. Already precessed clusters get skipped internally.
    """

    def __init__(self, args, job_start_cluster: int):
        self.args = args
        self.job_start_cluster = job_start_cluster
        random.seed(args.seed)

    def _contains_duplicates(self, arr):
        return len(np.unique(arr)) != len(arr)

    def _contains_duplicates(self, arr):
        # 使用集合来检测重复项
        seen = set()
        duplicates = set()
        for item in arr:
            if item in seen:
                duplicates.add(item)
            else:
                seen.add(item)
        if duplicates:
            print("Duplicates found:", duplicates)
            return True
        return False

    def semdedup(self, cluster, cluster_reps, device):
        st = time.time()
        ## -- compute pairwise cos sim between cluster items, then replace to diagonal with zeros to ignore self similarity
        cluster_reps.to(device)

        norms = torch.norm(cluster_reps, dim=1, keepdim=True)
        # 归一化每一行
        cluster_reps = cluster_reps / norms

        pair_w_sim_matrix = cluster_reps @ (cluster_reps.T)
        print(pair_w_sim_matrix)
        del cluster_reps
        pair_w_sim_matrix.fill_diagonal_(0.0)
        assert pair_w_sim_matrix.shape[0] == pair_w_sim_matrix.shape[1]

        ## -- get paths to cluster i images
        image_urls = cluster[:, 0]

        ## -- make sure all the paths are unique this ensure that the duplicates are really stored many time times on memory
        # try:
        #     assert not self._contains_duplicates(image_urls)
        # except:
        #     print('1')

        ## -- We need upper tringular matrix because (1)we don't need to look at self sim (always=1) (2)we need the compinations not permutations

        triu_sim_mat = torch.triu(pair_w_sim_matrix, diagonal=1)

        ## -- if the max sim between one example and any other example is > 1-eps, remove this example
        M = torch.max(triu_sim_mat, dim=0)[0].cpu()
        print(f"Step time: {time.time()-st}(s)")
        print('===================')
        print(M)
        print('===================')
        return M

    def _process_shard(self, start_cluster: int, end_cluster: int):
        # print("SemDeDup params: ", self.args)
        st = time.time()

        embs = init_memmap_embs(
            self.args.embs_memory_loc, self.args.dataset_size, self.args.emd_size
        )

        step_time = []

        for cluster_id in tqdm(range(start_cluster, end_cluster)):
            step_st = time.time()

            df_file_loc = os.path.join(
                self.args.save_loc, f"cluster_{cluster_id}.pkl"
            )

            if os.path.exists(df_file_loc):  # and os.path.exists(dict_file_loc):
                print(f"{df_file_loc} exists, moving on")
                continue

            ## -- load cluster i representations
            cluster_i = np.load(
                os.path.join(
                    self.args.sorted_clusters_path, f"cluster_{cluster_id}.npy"
                )
            )
            # 1) store cluster size
            cluster_size = cluster_i.shape[0]
            print("cluster_size: ", cluster_size)

            if cluster_size == 1:
                points_to_remove_df = pd.DataFrame()
                points_to_remove_df["indices"] = [0]
                for eps in self.args.eps_list:
                    ## We need to remove a point from the dataset when its pairwise similarity to other point is > 1-ebs
                    points_to_remove_df[f"eps={eps}"] = [False]
                if self.args.save_loc != "":
                    ## --save df
                    with open(df_file_loc, "wb") as file:
                        pickle.dump(points_to_remove_df, file)
                print("DONE cluster_id ", cluster_id)
                continue

            ## -- By default, we keep hard examples from groups
            clutser_items_indices = list(range(cluster_size))
            ## -- OR: shuffle cluster to keep random example from each group
            if self.args.which_to_keep.lower() == "random":
                random.shuffle(clutser_items_indices)
                cluster_i = cluster_i[clutser_items_indices]
            ## -- OR: reverse cluster to keep easy examples
            if self.args.which_to_keep.lower() == "easy":
                clutser_items_indices = clutser_items_indices[::-1]
                cluster_i = cluster_i[clutser_items_indices]

            ## -- indices for cluster items in the dataset
            cluster_ids = cluster_i[:, 1].astype("int32")
            cluster_reps = embs[cluster_ids]
            cluster_reps = torch.tensor(cluster_reps)

            M = self.semdedup(cluster_i, cluster_reps, self.args.device)

            points_to_remove_df = pd.DataFrame()
            points_to_remove_df["indices"] = clutser_items_indices

            for eps in self.args.eps_list:
                ## -- 5) We need to remove a point from the dataset when its pairwise similarity to other point is > 1-ebs
                eps_points_to_remove = M > 1 - eps
                points_to_remove_df[f"eps={eps}"] = eps_points_to_remove

            if self.args.save_loc != "":
                ## --save df
                with open(df_file_loc, "wb") as file:
                    pickle.dump(points_to_remove_df, file)

            # step_time.append_cluster(time.time() - step_st)
            step_time.append(time.time() - step_st)
            print("DONE cluster: ", cluster_id)

        # print(
        #     f"DONE in {((time.time()-st)/60):.2f} minutes, Average Step time {(sum(step_time)/len(step_time)):.2f}(s)"
        # )
        return

   
    def __call__(self):
        # pp = pprint.PrettyPrinter(indent=4)
        # pp.pprint(vars(self.args))
        job_start_cluster = self.job_start_cluster

        print(
            f"This job will process clusters {job_start_cluster} to  {min(self.args.num_clusters, job_start_cluster+self.args.clusters_per_job)}"
        )

        # job_env = submitit.JobEnvironment()

        # print(f"There are {job_env.num_tasks} tasks in this job")

        # print(f"I'm the task #{job_env.local_rank} on node {job_env.node}")
        # print(f"I'm the task #{job_env.global_rank} in the job")

        ## divide clusters across tasks (cpus)
        num_clusters_per_task = 1 #int(
        #     math.ceil(self.args.clusters_per_job / job_env.num_tasks)
        # )
        task_rank = 0
        start_cluster = job_start_cluster + task_rank * num_clusters_per_task
        end_cluster = job_start_cluster + (task_rank + 1) * num_clusters_per_task
        end_cluster = min(self.args.num_clusters, end_cluster)
        end_cluster = min(end_cluster, job_start_cluster + self.args.clusters_per_job)
        print(
            f"This task will process {num_clusters_per_task} clusters: cluster {start_cluster} to cluster {end_cluster}"
        )
        print(
            f"This task will process cluster {start_cluster} to cluster {end_cluster}"
        )

        self._process_shard(start_cluster, end_cluster)


In [2]:
import numpy as np

In [5]:
test_data = np.load('630/sorted_clusters/cluster_0.npy')
test_data

array([['/hpc2hdd/home/xzeng159/projects/LLaVA/playground/data/unichart-pretrain-data/content/images/0652d757c19bc62a3945b0dec17239a3_Y29sbGFib3JhdGlvbi5pZGZnLmlkYWhvLmdvdgkxNjQuMTY1LjY2Ljkx-0-3.png',
        '550140', '0.9112241864204407', '0'],
       ['/hpc2hdd/home/xzeng159/projects/LLaVA/playground/data/unichart-pretrain-data/content/images/OWID_Line_29975515006620.png',
        '84687', '0.910896360874176', '0'],
       ['/hpc2hdd/home/xzeng159/projects/LLaVA/playground/data/unichart-pretrain-data/content/images/OWID_Line_29975515006623.png',
        '84688', '0.9057799577713013', '0'],
       ...,
       ['/hpc2hdd/home/xzeng159/projects/LLaVA/playground/data/unichart-pretrain-data/content/images/OWID_Bar_16146648007068.png',
        '48271', '0.771998405456543', '0'],
       ['/hpc2hdd/home/xzeng159/projects/LLaVA/playground/data/unichart-pretrain-data/content/images/OWID_Bar_16146648007067.png',
        '48270', '0.7715266942977905', '0'],
       ['/hpc2hdd/home/xzeng159/proje

In [10]:
import math
import yaml


save_dir = './sample_files'
embs_memory_loc = f'{save_dir}/embeddings_memmap.dat'
save_loc = f"{save_dir}/results"
os.makedirs(save_loc, exist_ok=True)
sorted_clusters_path = f'{save_dir}/sorted_clusters'
eps_list = [0.0001,0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.4, 0.5, 0.9, 0.99]
which_to_keep = 'hard'
device = 'cpu'
clusters_per_job = 1
num_clusters = 100
# dataset_size, emb_size  = all_features.shape
dataset_size, emb_size  = (611934, 512)

# 计算总的jobs数量
total_jobs = math.ceil(num_clusters / clusters_per_job)

# 定义参数结构
from argparse import Namespace
args = Namespace(
    embs_memory_loc=embs_memory_loc,
    save_loc=save_loc,
    sorted_clusters_path=sorted_clusters_path,
    eps_list=eps_list,
    which_to_keep=which_to_keep,
    device=device,
    dataset_size=dataset_size,
    emd_size=emb_size,
    seed=1234,
    clusters_per_job=clusters_per_job,
    num_clusters=num_clusters
)

# 逐个处理cluster
for job in range(total_jobs):
    print('STARTING JOB: ', job)
    exp = SemDeDupJob(
        args=args,
        job_start_cluster=job * clusters_per_job
    )
    exp()

STARTING JOB:  0
This job will process clusters 0 to  1
This task will process 1 clusters: cluster 0 to cluster 1
This task will process cluster 0 to cluster 1


  0%|          | 0/1 [00:00<?, ?it/s]

cluster_size:  5419
tensor([[1.0000, 0.2436, 0.2289,  ..., 0.2860, 0.2606, 0.2851],
        [0.2436, 1.0000, 0.9584,  ..., 0.2763, 0.2677, 0.1968],
        [0.2289, 0.9584, 1.0000,  ..., 0.3116, 0.2863, 0.1976],
        ...,
        [0.2860, 0.2763, 0.3116,  ..., 1.0000, 0.9350, 0.6797],
        [0.2606, 0.2677, 0.2863,  ..., 0.9350, 1.0000, 0.6628],
        [0.2851, 0.1968, 0.1976,  ..., 0.6797, 0.6628, 1.0000]])
Step time: 0.17948198318481445(s)
tensor([0.0000, 0.2436, 0.9584,  ..., 0.9625, 0.9519, 0.9793])


100%|██████████| 1/1 [00:00<00:00,  4.54it/s]


DONE cluster:  0
STARTING JOB:  1
This job will process clusters 1 to  2
This task will process 1 clusters: cluster 1 to cluster 2
This task will process cluster 1 to cluster 2


...

tensor([[1.0000, 0.9738, 0.7950,  ..., 0.3494, 0.2563, 0.3078],
        [0.9738, 1.0000, 0.7953,  ..., 0.3482, 0.2479, 0.2960],
        [0.7950, 0.7953, 1.0000,  ..., 0.3396, 0.2884, 0.2938],
        ...,
        [0.3494, 0.3482, 0.3396,  ..., 1.0000, 0.7344, 0.8098],
        [0.2563, 0.2479, 0.2884,  ..., 0.7344, 1.0000, 0.7655],
        [0.3078, 0.2960, 0.2938,  ..., 0.8098, 0.7655, 1.0000]])
Step time: 0.10306119918823242(s)
tensor([0.0000, 0.9738, 0.7953,  ..., 0.8978, 0.8538, 0.8787])
DONE cluster:  98
STARTING JOB:  99
This job will process clusters 99 to  100
This task will process 1 clusters: cluster 99 to cluster 100
This task will process cluster 99 to cluster 100


  0%|          | 0/1 [00:00<?, ?it/s]

cluster_size:  1761


100%|██████████| 1/1 [00:00<00:00, 10.80it/s]

tensor([[1.0000, 0.0201, 0.1805,  ..., 0.2144, 0.2227, 0.2322],
        [0.0201, 1.0000, 0.2156,  ..., 0.3467, 0.3551, 0.3477],
        [0.1805, 0.2156, 1.0000,  ..., 0.2938, 0.3039, 0.3225],
        ...,
        [0.2144, 0.3467, 0.2938,  ..., 1.0000, 0.9821, 0.9527],
        [0.2227, 0.3551, 0.3039,  ..., 0.9821, 1.0000, 0.9523],
        [0.2322, 0.3477, 0.3225,  ..., 0.9527, 0.9523, 1.0000]])
Step time: 0.0031685829162597656(s)
tensor([0.0000, 0.0201, 0.2156,  ..., 0.9873, 0.9936, 0.9827])
DONE cluster:  99


In [ ]:
import os
import numpy as np
import pandas as pd


label_df = pd.read_csv('data_label.csv')

label_dict = label_df.set_index('img_id')['label'].to_dict()

os.makedirs('stratums', exist_ok=True)

for cluster_num in range(100):
    cluster_path = f'sorted_clusters/cluster_{cluster_num}.npy'
    cluster_data = np.load(cluster_path, allow_pickle=True)

    stratums = {}

    # for each cluter
    for item in cluster_data:
        img_path, img_id, similarity, cluster_id = item

        label = label_dict.get(img_id, None)
        if label is not None:
            new_item = [img_path, img_id, similarity, cluster_id, label]

            # split to stratums
            if label not in stratums:
                stratums[label] = []
            stratums[label].append(new_item)

    # save stratums files
    for label, stratum_data in stratums.items():
        stratum_path = f'stratums/cluster_{cluster_num}_stratum_{label}.npy'
        np.save(stratum_path, stratum_data)

print("Stratum files have been generated and saved.")


In [4]:
import os
from tqdm import tqdm
import pickle
import numpy as np


def extract_pruned_data(
    sorted_clusters_path,
    semdedup_pruning_tables_path,
    eps,
    num_clusters,
    output_txt_path,
    retreive_kept_samples=True,
):

    ## -- list of paths to the examples we want to keep/remove.
    example_paths = []

    for cluster_id in tqdm(range(0, num_clusters)):

        cluster_i = np.load(
            os.path.join(sorted_clusters_path, f"cluster_{cluster_id}.npy")
        )
        with open(
            f"{semdedup_pruning_tables_path}/cluster_{cluster_id}.pkl", "rb"
        ) as file:
            semdedup_pruning_tables = pickle.load(file)

        ## -- See which examples to keep/remove from this cluster.
        ## -- Use retreive_kept_samples=True when kept dataset size <= 50%. This will return a smaller output text file,
        ## -- semdedup_pruning_tables contain True values for the examples to be removed.
        images_to_keep_or_remove = semdedup_pruning_tables[f"eps={eps}"][
            semdedup_pruning_tables[f"eps={eps}"] == (not retreive_kept_samples)
        ].index.to_numpy()
        if "indices" in semdedup_pruning_tables.columns:
            cluster_i = cluster_i[semdedup_pruning_tables["indices"]]
        ## -- retrieve only the examples we want and add to the list.
        dedup_cluster = cluster_i[images_to_keep_or_remove]
        example_paths += dedup_cluster[:, IMAGE_NAME_INDEX].astype("<U200").tolist()

    with open(output_txt_path, "w") as fp:
        fp.write("\n".join(example_paths))

    print(f"DONE saving {len(example_paths)} image paths")

    return


In [14]:
output_txt_path = f'{save_dir}/test.txt'
semdedup_pruning_tables_path = f'{save_dir}/results'
sorted_stratums_paths = f'{save_dir}/sorted_stratums'
eps = 0.32
num_clusters = 100
extract_pruned_data(sorted_stratums_paths, semdedup_pruning_tables_path, eps, num_clusters, output_txt_path, retreive_kept_samples=True)

100%|██████████| 100/100 [00:01<00:00, 77.31it/s]

DONE saving 100540 image paths
